In [2]:
import dolfin as dolf
from math import pi
import numpy as np
from helmholtz_pkg.passive_flame import PassiveFlame
from helmholtz_pkg.flame_transfer_function import n_tau
from helmholtz_pkg.active_flame import ActiveFlame
from helmholtz_pkg.eigensolvers import fixed_point_iteration_pep
from helmholtz_pkg.eigenvectors import normalize_eigenvector
from helmholtz_pkg.mshr import *
import params
import matplotlib.pyplot as plt

In [3]:
def mshr():

    geometry = MeshXDMF('MeshDir/pipe', write_xdmf_file=True)
    geometry()
    mesh = geometry.mesh
    boundaries = geometry.boundaries
    subdomains = geometry.subdomains
    
    return mesh, boundaries, subdomains



In [4]:
degree = 1

mesh, boundaries, subdomains = mshr()

boundary_conditions = {1: {'Robin': params.Y_in},  # inlet
                       2: {'Robin': params.Y_out}}  # outlet



In [5]:
radius = 0.047/2

L = 1
total_volume = np.pi*radius**2*L
print("Total volume of tube is: ",total_volume)

L_flame = params.a_f*2
flame_volume = np.pi*radius**2*L_flame
print("Flame region volume is: ",flame_volume)


def calculate_volume_subdomain(mesh, subdomains):
    V = dolf.FunctionSpace(mesh, 'CG', 2)
    u = dolf.interpolate(dolf.Expression('1',degree = 0), V, )

    dV = dolf.Measure('dx', subdomain_data = subdomains)
    print("Flame Volume: ", dolf.assemble(u*dV(0)))
    
def calculate_whole_domain(mesh, subdomains):
    volume = dolf.MeshFunction('size_t', mesh, mesh.topology().dim())
    V = dolf.FunctionSpace(mesh, 'CG', 1)
    u = dolf.interpolate(dolf.Expression('1',degree = 0), V, )
    #volume.set_all(999)
    dV = dolf.Measure('dx', subdomain_data = subdomains)
    print("Total Volume: ", dolf.assemble(u*dV(999)+u*dV(0)))

print("\nCalculated Mesh Volume:\n")
calculate_whole_domain(mesh, subdomains)
calculate_volume_subdomain(mesh, subdomains)


Total volume of tube is:  0.0017349445429449633
Flame region volume is:  8.674722714724817e-05

Calculated Mesh Volume:

Total Volume:  0.0017253237225347729
Flame Volume:  8.630137244752388e-05


In [ ]:
V = dolf.FunctionSpace(mesh, 'CG', 1)
u = dolf.interpolate(dolf.Expression('1',degree = 0), V, )


lc = 5e-3, 

| Element size(m) | Flame Region Volume(m3) | Total Volume(m3) | Simulation Time(s)|
| 1e-2 | 7.5096e-05 | 0.0017253 | 776.210 
| 5e-3 | 6.6265e-05 | 0.0016901 | 15.870 
| Stretched | Gaussian | .843 | ---- |